https://www.metacritic.com/game 에서 크롤링 예정

In [1]:
import re
import requests
import pandas as pd
import time
from bs4 import BeautifulSoup

헤더부분 유저에이전트,레퍼러 예제

In [2]:
hur={'User-Agent':'Mozilla','referer':'https://www.metacritic.com/game/playstation-5/elden-ring'}
url='https://www.metacritic.com/game/playstation-5/elden-ring'
req=requests.get(url,headers=hur)
bsoup=BeautifulSoup(req.content,'html.parser',from_encoding='utf-8') #beautifulsoup객체선언
q1=bsoup.find("div",{"class":"details main_details"})
temp=bsoup.find("span",{"class":"blurb blurb_expanded"}).text
print(temp)

A New World Created By Hidetaka Miyazaki And George R. R. Martin. ELDEN RING, developed by FromSoftware, Inc. and BANDAI NAMCO Entertainment Inc., is a fantasy action-RPG adventure set within a world created by Hidetaka Miyazaki creator of the influential DARK SOULS video game series; and George R.R. Martin author of The New York Times best-selling fantasy series, A Song of Ice and Fire. Danger and discovery lurk around every corner in FromSoftware's largest game to-date.  Hidetaka Miyazaki, President and Game Director of FromSoftware Inc. Known for directing critically-acclaimed games in beloved franchises including Armored Core, Dark Souls, and Sekiro: Shadows Die Twice.  George R.R. Martin is the #1 New York Times bestselling author of many novels, including the acclaimed series A Song of Ice and Fire - A Game of Thrones, A Clash of Kings, A Storm of Swords, A Feast For Crows, and A Dance with Dragons. As a writer-producer, he has worked on The Twilight Zone, Beauty and the Beast, a

이전까지 크롤링했던 진도부터 하기

정해진 플랫폼에 따라 크롤링하는 함수

In [ ]:
def crawl_page(url):
    ret={}
    chk=True
    print(url)
    while(chk):
        req=requests.get(url,headers={'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36','referer':url}) 
        if(req==None): 
            print("reqNone")
            continue
        bsoup=BeautifulSoup(req.text,'lxml')
        time.sleep(1.5)#크롤링 속도 제한
        if(bsoup==None):
            print("bsoupNone")
            continue
        bmetascore=bsoup.find("span",{"itemprop":"ratingValue"})
        if(bmetascore==None):
            time.sleep(4)
            print(bsoup)
            print("bemtaNone")
            continue
        bsummary=bsoup.find("span",class_="blurb blurb_expanded")
        if(bsummary==None):
            print("bsumm")
            bsummary="None"
        else:
            if(bsummary.string==None):
                bsummary=bsoup.find("span",class_="blurb blurb_collapsed")
                if(bsummary.string!=None):
                    bsummary=bsummary.string
                if(bsummary.string!=None):
                    bsummary=bsummary.string
            else:
                bsummary=bsummary.string
            print(bsummary)
        btitle=bsoup.find("div",{"class":"product_title"}).find("h1")
        if(btitle==None): 
            print("btitleNone")
            continue
        buserscore=bsoup.find("div",class_=re.compile("metascore_w user large game"))
        if(buserscore==None): 
            print("buserNone")
            continue
        bpub=bsoup.find("li",class_="summary_detail publisher").find("a")
        if(bpub==None):
            print("bpubNone")
            continue
        bdev=(bsoup.find("li",class_="summary_detail developer"))
        if(bdev==None):
            print("bdevNone")
            continue
        else:
            bdev=bdev.find("a")
        bgenres=(bsoup.find("li",class_="summary_detail product_genre"))
        if(bgenres==None):
            print("bgenreNone")
            continue
        bgenres=bgenres.find_all("span",class_="data")
        sgenres=""
        for x in range(len(bgenres)):
            sgenres=sgenres+str(bgenres[x].string)+"|"
        bgrating=bsoup.find("li",class_="summary_detail product_rating")
        if(bgrating==None):
            print("bgratingNone")
            bgrating="None"
        else:
            bgrating=bgrating.find("span",class_="data").string
        brelease=bsoup.find("li",class_="summary_detail release_data")
        brelease=brelease.find("span",class_="data")
        print(str(btitle.string)," ",str(brelease.string)," ",str(bmetascore.string)," ",str(buserscore.string)," ",str(bpub.string)[57:-53]," ",str(bdev.string)," ",sgenres," ",bgrating)
        ret={"title":str(btitle.string),"metascore":str(bmetascore.string),"userscore":str(buserscore.string),"publisher":str(bpub.string)[57:-53],"developer":str(bdev.string),"genres":sgenres,"rating":str(bgrating),"summary":str(bsummary),"release_date":str(brelease.string)}
        chk=False
    return ret
    
def get_page(url):
    req=requests.get(url,headers={'User-Agent':'Mozilla','referer':url})  
    bsoup=BeautifulSoup(req.content,'html.parser',from_encoding='utf-8')
    bsoup2=bsoup.find_all("td",{"class":"clamp-summary-wrap"})
    lpd=pd.DataFrame(columns={"title","release_date","publisher","developer","genres","rating","summary","metascore","userscore"})
    cnt=0
    for x in bsoup2:
        y=x.find("a")["href"]
        y=y[:-15]
        df_dic=pd.DataFrame([crawl_page("https://www.metacritic.com/"+y)])
        lpd = pd.concat([lpd, df_dic], ignore_index=True)
    return lpd
        
def crawl_direct(platform,df):
    url="https://www.metacritic.com/browse/games/release-date/available/"+platform+"/metascore?page="
    req=requests.get(url,headers={'User-Agent':'Mozilla','referer':url+'0'})
    bsoup=BeautifulSoup(req.content,'html.parser',from_encoding='utf-8')
    lpage_soup=bsoup.find("li",{"class":"last_page"})
    lpage2_soup=lpage_soup.find("a",{"class":"page_num"})
    lpagenum=lpage2_soup.text
    try:
      lpd=pd.read_csv(platform+'.csv')
    except:
        lpd=pd.DataFrame(columns={"title","release_date","publisher","developer","genres","rating","summary","metascore","userscore"})
    proc=int(len(lpd)/100)
    for x in range(proc,int(lpagenum)):  
        print(url+str(x))
        page=get_page(url+str(x))
        page=pd.concat([page,lpd],ignore_index=True)
        page.to_csv(platform+'.csv',sep=',',na_rep='NaN',float_format = '%.2f',index = False)
crawl_direct('ps4',1)

https://www.metacritic.com/browse/games/release-date/available/ps4/metascore?page=1
https://www.metacritic.com//game/playstation-4/spelunky-2
Cenários e gameplay repleto de segredos, física aprimorada, novos caminho, música bem trabalhada, level design procedural de encher os olhos. Uma run nunca será igual a outra, te obrigando a melhorar como jogador. Pra quem busca um desafio, aqui está!
Spelunky 2   Sep 15, 2020   87   7.8   Mossmouth   Mossmouth   Action|Platformer|2D|   T
https://www.metacritic.com//game/playstation-4/fallout-4
Even decades after Bethesda started making games, there are still only a handful of RPGs that have ever offered anywhere CLOSE to the amount
Fallout 4   Nov 10, 2015   87   6.8   Bethesda Softworks   Bethesda Game Studios   General|Role-Playing|Western-Style|   M
https://www.metacritic.com//game/playstation-4/uncharted-the-nathan-drake-collection
From the storytellers at Naughty Dog, comes the epic that altered adventure storytelling, rebuilt by Bluepoint 

Ni no Kuni: Wrath of the White Witch Remastered = Journey back to the other world in Ni no Kuni: Wrath of the White Witch. LEVEL-5's classic tale returns better than ever. Join Oliver as he embarks on an adventure through a world inhabited by new friends and ferocious foes alike in the hopes
Ni no Kuni: Wrath of the White Witch Remastered   Sep 20, 2019   86   8.2   Bandai Namco Games   Level 5   Role-Playing|Japanese-Style|   E10+
https://www.metacritic.com//game/playstation-4/guilty-gear-xrd--revelator-
Best fighting game in the market  so far. BUY IT NOW !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Guilty Gear Xrd -REVELATOR-   Jun  7, 2016   86   8.0   Aksys Games   Arc System Works   Action|Fighting|2D|   T
https://www.metacritic.com//game/playstation-4/dying-light-the-following---enhanced-edition
Enjoy the definitive Dying Light experience with the new Legend system, improved visuals, significant gameplay enhancements, and mor

To battle the Darkness and the Taken army, players get brand new subclasses with new supers. Warlocks now harness the power of an electric storm, Hunters carve a bow formed from gravitational void energy, and Titans summon a flaming hammer that can sear their adversaries from a range or deliver a devastating melee impact.
Destiny: The Taken King   Sep 15, 2015   86   6.3   Activision   Bungie   Action|Shooter|First-Person|Arcade|   T
https://www.metacritic.com//game/playstation-4/ender-lilies-quietus-of-the-knights
Long ago, in the distant reaches of Land's End, an unforeseen rainfall transformed all living things into frenzied undead monstrosities known as the Blighted. Faced with a calamity far beyond their understanding, the kingdom fell into ruin with no end to the accursed rain in sight.
ENDER LILIES: Quietus of the Knights   Jun 22, 2021   86   7.6   Binary Haze Interactive   Live Wire Films   Action|Action Adventure|Platformer|Open-World|None|Metroidvania|   T
https://www.metacr

Wonderful game that finally came to PS4. Must buy for everyone who are at least a bit interested in metrovania style games. Gem in EA filled **** franchise-game industry.
Hollow Knight: Voidheart Edition   Sep 25, 2018   85   8.9   Team Cherry   Team Cherry   Action|Action Adventure|General|Platformer|None|Metroidvania|   E10+
https://www.metacritic.com//game/playstation-4/life-is-strange
Further proof that video games can tell emotionally engaging stories and present relatable characters. Superb voice acting, beautiful art design, a smooth score, and good use of licensed music all help to make this story game one worth playing. If you are looking to take a break from the usual RPGs and shooters, then I recommend you start with Life is Strange.
Life is Strange   Oct 20, 2015   85   8.6   Square Enix   DONTNOD Entertainment   Sci-Fi|Action Adventure|General|   M
https://www.metacritic.com//game/playstation-4/steamworld-dig-2
The upgrades played a big role in the original Dig. This time 

Freaking awesome. Any true ballplayer would appreciate the game they've put together with The Show. Didn't play another game for a full 7 months after it's release... it's THAT good.
MLB The Show 17   Mar 28, 2017   85   6.9   Sony Interactive Entertainment   Sony Interactive Entertainment   Sports|Team|Baseball|Sim|   E
https://www.metacritic.com//game/playstation-4/destiny-2
In Destiny 2, the last safe city on Earth has fallen and lays in ruins, occupied by a powerful new enemy and his elite army, the Red Legion. Every player creates his own character called a “Guardian,” humanity’s chosen protectors.  As a Guardian in Destiny 2, players must master new abilities and weapons to reunite the city’s forces, stand together and fight back to reclaim their home. In Destiny 2 players embark on a fresh story filled with new destinations around our solar system to explore, and an expansive amount of activities to discover.  There is something for almost every type of gamer in Destiny 2, inclu

Meet Quill, a young mouse with dreams of greatness beyond the confines of her quiet settlement. One day while exploring the woods, she finds a mysterious stone and an ancient magic is awakened. Dramatic events quickly unfold as a distant evil extends its reach. With her uncle now in grave danger, Quill must embark on an epic journey—and she needs you there by her side. Together, you'll travel to forgotten realms, solve challenging puzzles, and battle menacing enemies. Alone, no one can conquer what you're up against. But united, you just may defeat even the darkest of villains. Immerse yourself in the world of Moss to help write the story of a hero in the making. [Playstation.com]
Moss   Feb 27, 2018   85   8.1   Sony Interactive Entertainment   Polyarc   Action Adventure|General|   E10+
https://www.metacritic.com//game/playstation-4/tomb-raider-definitive-edition
Very well crafted game. A bit short but well worth the price on sale. If you played the old Tomb raiders you know the game 

HITMAN 3 is the dramatic conclusion to the World of Assassination trilogy and takes players around the world on a globetrotting adventure to sprawling sandbox locations. Agent 47 returns as a ruthless professional for the most important contracts of his entire career.</span>
Hitman 3   Jan 20, 2021   84   8.0   Io Interactive   Io Interactive   Action Adventure|General|   M
https://www.metacritic.com//game/playstation-4/kingdom-hearts-hd-i5-+-ii5-remix
Were you like me; a fan of Kingdom Hearts 1, 2 and Re:Chain of Memories but didn't really have a chance to play the plot-relevant portable entries of this franchise? Worry not! This compilation in combination with 'Kingdom Hearts 2.8: Final Chapter Prologue' offers the perfect way to catch up on the story of one of the most beloved series in gaming history up to this point for quite a decent price. There were some issues when the game first launched mainly due to the 60 frames per second upgrade (excessively fast animations, save crashes

DiRT Rally 2.0 dares you to carve your way through a selection of iconic rally locations from across the globe, in the most powerful off-road vehicles ever made, knowing that the smallest mistake could end your stage. You will need to rely on your instincts with the most immersive and truly
DiRT Rally 2.0   Feb 26, 2019   84   5.9   Codemasters   Codemasters   Racing|Simulation|Automobile|   E
https://www.metacritic.com//game/playstation-4/deus-ex-mankind-divided
The action of Deus Ex: Mankind Divided takes place in 2029, a couple of years after the events of Human Revolution and the infamous ‘Aug Incident’ in Panchaea that resulted in the deaths of millions at the hands of those who had installed augmentations. This event has caused a massive divide between those who have augmentations and those who don't. Amidst this emotional turmoil are various factions aiming to manipulate the public by twisting public opinion of augmentation to advance their own agenda and hide the truth of what 

I just got a PS4 and the first game I played was Spiderman Miles Morales. I am a huge Spiderman fan and this game finally brought him justice in the video game world since Spider Man 2. This is my favourite game of all time and the graphics are stunning even on a PS4. Definitely worth the 60 bucks.
Marvel's Spider-Man: Miles Morales   Nov 12, 2020   84   7.6   PlayStation Studios   Insomniac Games   Action Adventure|Open-World|   T
https://www.metacritic.com//game/playstation-4/f1-2019
The official videogame of the 2019 FIA Formula One World Championship, F1 2019 challenges you to Defeat your Rivals in the most ambitious F1 game in Codemasters' history. The game is released a full two months earlier during the Formula 1 myWorld Grosser Preis von Osterreich 2019 weekend
F1 2019   Jun 25, 2019   84   7.7   Codemasters   Codemasters   Racing|Simulation|Automobile|   E
https://www.metacritic.com//game/playstation-4/lara-croft-go
Lara Croft Go is a cross between a puzzle game and turn based

The Overlord is dead. The Netherworld is in turmoil. Time to kick some ass! Netherworld Prince Laharl awakens to fight for his throne and it's up to him and some unlikely allies to defend what is rightfully his. Together with his devious vassal Etna, the angel trainee Flonne, and a whole cavalcade of explosive characters, Laharl will slash, shoot, and punch his way to the top. Build your squad to level 9999 and unleash crazy special skills, team attacks, and spells to deal millions of points of damage in this zany high-definition remake of an RPG classic.
Disgaea 1 Complete   Oct  9, 2018   83   7.5   NIS America   Nippon Ichi Software   Strategy|Turn-Based|Tactics|   T
https://www.metacritic.com//game/playstation-4/pro-evolution-soccer-2018
Real Touch+ adds a new dimension to ball control, with full body touch allowing various parts of the body to control the ball based on the unpredictability of the ball movement. Set pieces have been reworked with a new Free Kick and Penalty Kick sy